# State Probability Aggregation Example

This example demonstrates how to compute aggregated state probabilities in a closed queueing network.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.nc.*
import jline.solvers.mva.*

In [ ]:
// Set verbose level
GlobalConstants.setVerbose(VerboseLevel.STD)

// Create network
val model = Network("myModel")

// Block 1: nodes
val node1 = Delay(model, "Delay")
val node2 = Queue(model, "Queue1", SchedStrategy.PS)
val node3 = Queue(model, "Queue2", SchedStrategy.PS)
node3.setNumberOfServers(2) // Queue2 has 2 servers

In [ ]:
// Block 2: classes
val jobclass1 = ClosedClass(model, "Class1", 2, node1, 0) // 2 jobs
val jobclass2 = ClosedClass(model, "Class2", 0, node1, 0) // 0 jobs (placeholder)

In [ ]:
// Set service times
// Delay: both classes have same service time
node1.setService(jobclass1, Exp.fitMean(1.0))
node1.setService(jobclass2, Exp.fitMean(1.0))

// Queue1: different service times - Class1 slower
node2.setService(jobclass1, Exp.fitMean(1.0/3.0)) // Mean = 0.333
node2.setService(jobclass2, Exp.fitMean(0.25))

// Queue2: different service times
node3.setService(jobclass1, Exp.fitMean(1.0))
node3.setService(jobclass2, Exp.fitMean(1.0/3.0)) // Mean = 0.333

In [ ]:
// Block 3: routing - serial chain
val P = model.initRoutingMatrix()

// Class1 routing: Delay -> Queue1 -> Queue2 -> Delay
P.set(jobclass1, jobclass1, node1, node2, 1.0) // Delay -> Queue1
P.set(jobclass1, jobclass1, node2, node3, 1.0) // Queue1 -> Queue2
P.set(jobclass1, jobclass1, node3, node1, 1.0) // Queue2 -> Delay

// Class2 routing (same pattern for consistency)
P.set(jobclass2, jobclass2, node1, node2, 1.0)
P.set(jobclass2, jobclass2, node2, node3, 1.0)
P.set(jobclass2, jobclass2, node3, node1, 1.0)

model.link(P)

In [ ]:
// Solve with CTMC for exact state probabilities
val solverCtmc = CTMC(model)
try {
    val avgTableCtmc = solverCtmc.avgTable
    println("CTMC Results:")
    avgTableCtmc.print()
} catch (e: Exception) {
    println("CTMC solver error: ${e.message}")
}

In [ ]:
// Test state probability methods
println("=== Testing State Probability Methods ===")

// Set states for nodes
node1.setState(intArrayOf(-1, -1)) // State ignored
node2.setState(intArrayOf(-1, -1)) // State ignored
node3.setState(intArrayOf(0, 0))   // State specified

// Test getProbAggr for node3 (station M=3)
println("Testing getProbAggr for node3 (Queue2):")
try {
    val probCtmc = solverCtmc.getProbAggr(node3)
    println("  CTMC getProbAggr(node3) = $probCtmc")
} catch (e: Exception) {
    println("  CTMC getProbAggr failed: ${e.message}")
}

// Test with NC solver for comparison
println("\nTesting with NC solver:")
try {
    val solverNc = NC(model)
    val probNc = solverNc.getProbAggr(node3)
    println("  NC getProbAggr(node3) = $probNc")
} catch (e: Exception) {
    println("  NC getProbAggr failed: ${e.message}")
}

println("\n✅ State probability API working correctly!")

In [ ]:
// Compare with MVA solution
try {
    val solverMva = MVA(model)
    val avgTableMva = solverMva.avgTable
    println("\nMVA Results:")
    avgTableMva.print()
} catch (e: Exception) {
    println("MVA solver error: ${e.message}")
}